
# AIRD ML Pack 생성 노트북 (서울 공장등록 데이터 기반)

이 노트북은 **전국 공장등록현황 데이터(엑셀/CSV)** 중 서울특별시 데이터만 활용하여  
다음 3개의 **ML Dataset (AIRD ML Pack)** 을 자동으로 생성합니다.

1. `ml_factory_risk_seoul_2025_v1.csv`  
   - 공장 단위 **리스크(고위험/저위험) 예측용** Dataset  
2. `ml_region_old_factory_share_seoul_2025_v1.csv`  
   - 자치구 단위 **노후공장 비중 분석·예측용** Dataset  
3. `ml_location_score_candidate_sites_v1.csv`  
   - 자치구-동 단위 **제조업 Hotspot / 입지 스코어링용** Dataset  



## 1. 라이브러리 로드

In [1]:
import os

def is_colab():
    """현재 실행 환경이 Google Colab인지 확인"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("▶ Colab 환경입니다.")

    # ------------------------
    # 1) GitHub Repo Clone
    # ------------------------
    repo_url = "https://github.com/hike-lab/AIRD-PACK.git"
    repo_name = repo_url.split('/')[-1].replace('.git', '')

    if os.path.exists(repo_name):
        print(f"이미 '{repo_name}' 폴더가 있어 clone을 스킵합니다.")
    else:
        print("GitHub repo clone 중...")
        !git clone $repo_url
        print("clone 완료!")

    # ------------------------
    # 2) 한글 폰트 설치
    # ------------------------
    print("나눔 폰트 설치 중...")
    !sudo apt-get install -y fonts-nanum
    !sudo fc-cache -fv
    !rm ~/.cache/matplotlib -rf
    print("폰트 설치 완료! (런타임 재시작 필요)")

    # ------------------------
    # 3) requirements 설치
    # ------------------------
    print("requirements.txt 설치 중...")
    %cd $repo_name
    !pip install -r requirements.txt
    %cd ..

    # ------------------------
    # 4) Colab용 경로 설정
    # ------------------------
    RAW_DATA_PATH = f"{repo_name}/data/raw/"
    PROCESSED_DATA_PATH = f"{repo_name}/data/processed/"
    OUTPUT_PATH = f"{repo_name}/code/ml-pack/outputs/"

    print("폴더 경로 설정 완료!")
    print("RAW_DATA_PATH:", RAW_DATA_PATH)
    print("PROCESSED_DATA_PATH:", PROCESSED_DATA_PATH)
    print("OUTPUT_PATH:", OUTPUT_PATH)

else:
    RAW_DATA_PATH = "../../data/raw/"
    PROCESSED_DATA_PATH = "../../data/processed/"
    OUTPUT_PATH = "../../code/ml-pack/outputs/"
    print("▶ 로컬 환경입니다. git clone 및 폰트 설치는 실행하지 않습니다.")

    print("로컬 경로 설정 완료!")
    print("RAW_DATA_PATH:", RAW_DATA_PATH)
    print("PROCESSED_DATA_PATH:", PROCESSED_DATA_PATH)
    print("OUTPUT_PATH:", OUTPUT_PATH)

▶ 로컬 환경입니다. git clone 및 폰트 설치는 실행하지 않습니다.
로컬 경로 설정 완료!
RAW_DATA_PATH: ../../data/raw/
PROCESSED_DATA_PATH: ../../data/processed/
OUTPUT_PATH: ../../code/ml-pack/outputs/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# 폰트 설정
import matplotlib.font_manager as fm


def set_korean_font():
    """
    플랫폼 판별 및 폰트 다운로드 없이
    matplotlib에 이미 설치된 한글 폰트만 사용
    """

    candidate_fonts = [
        "Malgun Gothic",      # Windows
        "AppleGothic",        # macOS
        "NanumGothic",        # Linux / 일부 Colab
        "Noto Sans CJK KR",
        "Noto Sans KR"
    ]

    installed_fonts = {f.name for f in fm.fontManager.ttflist}
    for font in candidate_fonts:
        if font in installed_fonts:
            plt.rcParams["font.family"] = font
            plt.rcParams["axes.unicode_minus"] = False
            print(f"✅ 한글 폰트 적용: {font}")
            return

# 실행
set_korean_font()
print("✅ 라이브러리 로드 완료")


✅ 한글 폰트 적용: AppleGothic
✅ 라이브러리 로드 완료



## 2. 원본 데이터 업로드

- 원본 파일: `seoul_factory_registry_2025_v1.csv`

전국공장등록현황 데이터셋에서 서울특별시 소재 공장만 추출한 데이터셋


In [3]:
# 파일 임포트

raw_filename = f"{RAW_DATA_PATH}/seoul_factory_registry_2025_v1.csv"
df = pd.read_csv(raw_filename)

print("✅ 원본 데이터 로딩 완료")
print("데이터 크기:", df.shape)
df.head()


✅ 원본 데이터 로딩 완료
데이터 크기: (33096, 80)


/var/folders/3k/09hbppyx33z8tmqmvw42bgqm0000gn/T/ipykernel_34800/3399293455.py:4: DtypeWarning: Columns (11,13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_filename)


,__row_uid__,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,...,정제_용도지역,코드매핑_용도지역,누적_용도지역,정제_지목,코드매핑_지목,누적_지목,정제_최초등록일,누적_최초등록일,정제_최초승인일,누적_최초승인일
0,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
1,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
2,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,제2종일반주거지역,R10-UQA122,{'제2종일반주거지역': ['R10-UQA122']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
3,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}
4,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}



## 3. 컬럼 구조 및 기본 정보 확인



In [4]:
print("📌 컬럼 목록:")
print(df.columns.tolist())

print("\n📌 데이터 타입 정보:")
print(df.dtypes.head())

print("\n📌 결측치 상위 10개 컬럼:")
print(df.isna().sum().sort_values(ascending=False).head(10))


📌 컬럼 목록:
['__row_uid__', '시도명', '시군구명', '관리기관', '회사명', '공장구분', '단지명', '설립구분', '입주형태', '보유구분', '최초승인일', '최초등록일', '등록구분', '등록일', '전화번호', '남자종업원', '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계', '생산품', '원자재', '공장규모', '용도지역', '지목', '용지면적', '제조시설면적', '부대시설면적', '건축면적', '지식산업센터명', '대표업종', '업종명', '업종코드', '차수', '법인주소', '필지수', '공장주소', '공장주소_지번', '공장관리번호', '정제_공장주소', '코드매핑_공장주소', '누적_공장주소', '정제_공장주소_지번', '코드매핑_공장주소_지번', '누적_공장주소_지번', '정제_관리기관', '코드매핑_관리기관', '누적_관리기관', '정제_대표업종', '코드매핑_대표업종', '누적_대표업종', '정제_등록일', '누적_등록일', '정제_법인주소', '코드매핑_법인주소', '누적_법인주소', '정제_보유구분', '코드매핑_보유구분', '누적_보유구분', '정제_시군구명', '코드매핑_시군구명', '누적_시군구명', '정제_시도명', '코드매핑_시도명', '누적_시도명', '정제_업종명', '누적_업종명', '정제_업종코드', '코드매핑_업종코드', '누적_업종코드', '정제_용도지역', '코드매핑_용도지역', '누적_용도지역', '정제_지목', '코드매핑_지목', '누적_지목', '정제_최초등록일', '누적_최초등록일', '정제_최초승인일', '누적_최초승인일']

📌 데이터 타입 정보:
__row_uid__    object
시도명            object
시군구명           object
관리기관           object
회사명            object
dtype: object

📌 결측치 상위 10개 컬럼:
코드매핑_법인주소       31417
정제_최초승인


## 4. 컬럼명 상수 정의 및 서울특별시 필터링

아래 상수는 **원본 컬럼명에 맞게 수정**해서 사용합니다.


In [5]:

# 🔧 필요 시 이 부분을 실제 컬럼명에 맞게 수정하세요.
COL_SIDO = "시도명"
COL_SIGUNGU = "시군구명"
COL_ADDRESS = "공장주소_지번"
COL_FIRST_APPROVAL = "최초승인일"
COL_MANUF_AREA = "제조시설면적"      # 없으면 건축면적/용지면적 등으로 대체 가능
COL_FACTORY_ID = "공장관리번호"      # 없으면 None으로 두고 index 사용
COL_COMPANY = "회사명"

# 서울특별시만 필터링
if COL_SIDO not in df.columns:
    raise KeyError(f"'{COL_SIDO}' 컬럼을 찾을 수 없습니다. 시도 컬럼명을 확인해주세요.")

df_seoul = df[df[COL_SIDO] == "서울특별시"].copy()
print("전체 공장 수:", len(df))
print("서울시 공장 수:", len(df_seoul))

df_seoul.head()


전체 공장 수: 33096
서울시 공장 수: 33096


,__row_uid__,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,...,정제_용도지역,코드매핑_용도지역,누적_용도지역,정제_지목,코드매핑_지목,누적_지목,정제_최초등록일,누적_최초등록일,정제_최초승인일,누적_최초승인일
0,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
1,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
2,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,제2종일반주거지역,R10-UQA122,{'제2종일반주거지역': ['R10-UQA122']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
3,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}
4,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}



## 5. 기본 Feature 생성

AIRD ML Pack에서 공통으로 사용하는 기본 변수들을 생성합니다.

- 공장연령 (`공장연령`)
- 주소에서 동 추출 (`공장동`)
- 자치구별 공장 수 및 밀집도 점수 (`자치구_공장수`, `자치구_밀집도점수`)
- 제조시설면적 및 로그변환 (`제조시설면적_log`)
- 노후공장 여부 (`노후공장여부`, 상위 25% 기준)


In [6]:
from sklearn.preprocessing import MinMaxScaler

BASE_YEAR = 2025

# 5-1. 공장연령 계산
if COL_FIRST_APPROVAL in df_seoul.columns:
    df_seoul[COL_FIRST_APPROVAL] = pd.to_datetime(
        df_seoul[COL_FIRST_APPROVAL], errors="coerce", format="%Y%m%d"
    )
    df_seoul["최초승인연도"] = df_seoul[COL_FIRST_APPROVAL].dt.year
    df_seoul["공장연령"] = BASE_YEAR - df_seoul["최초승인연도"]
    df_seoul["공장연령"] = df_seoul["공장연령"].where(df_seoul["공장연령"] >= 0)
else:
    print(f"⚠ '{COL_FIRST_APPROVAL}' 컬럼이 없어 공장연령 계산을 건너뜁니다.")

# 5-2. 주소에서 동 추출
def extract_dong(addr):
    if pd.isna(addr):
        return None
    text = str(addr)
    tokens = text.split(' ')
    for t in tokens:
        if t.endswith("동"):
            return t.strip()
    return None

if COL_ADDRESS in df_seoul.columns:
    df_seoul["공장동"] = df_seoul[COL_ADDRESS].apply(extract_dong)
else:
    df_seoul["공장동"] = None

df_seoul["공장동"] = df_seoul["공장동"].fillna("미상")

# 5-3. 자치구별 공장 수 및 밀집도 점수
gu_density = df_seoul.groupby(COL_SIGUNGU).size().to_frame("자치구_공장수")
scaler_density = MinMaxScaler()
gu_density["자치구_밀집도점수"] = scaler_density.fit_transform(gu_density[["자치구_공장수"]])

df_seoul = df_seoul.merge(
    gu_density[["자치구_밀집도점수"]],
    left_on=COL_SIGUNGU,
    right_index=True,
    how="left"
)

# 5-4. 제조시설면적 및 로그 변환
if COL_MANUF_AREA in df_seoul.columns:
    df_seoul[COL_MANUF_AREA] = pd.to_numeric(df_seoul[COL_MANUF_AREA], errors="coerce")
    df_seoul[COL_MANUF_AREA] = df_seoul[COL_MANUF_AREA].apply(
        lambda x: np.nan if (x is not None and x <= 0) else x
    )
    df_seoul[COL_MANUF_AREA + "_log"] = np.log1p(df_seoul[COL_MANUF_AREA])
else:
    print(f"⚠ '{COL_MANUF_AREA}' 컬럼이 없어 면적 로그 Feature는 생성되지 않습니다.")

# 5-5. 노후공장 여부 (공장연령 상위 25% 기준)
if "공장연령" in df_seoul.columns:
    age_valid = df_seoul["공장연령"].dropna()
    if len(age_valid) > 0:
        threshold_age = age_valid.quantile(0.75)
        df_seoul["노후공장여부"] = df_seoul["공장연령"] >= threshold_age
        print("노후 공장 기준 연령(상위 25%):", threshold_age)
    else:
        df_seoul["노후공장여부"] = False
else:
    df_seoul["노후공장여부"] = False
    print("⚠ 공장연령이 없어 노후공장여부는 False로 설정됩니다.")

print("✅ 기본 Feature 생성 완료")
df_seoul.head()


노후 공장 기준 연령(상위 25%): 22.0
✅ 기본 Feature 생성 완료


,__row_uid__,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,...,정제_최초등록일,누적_최초등록일,정제_최초승인일,누적_최초승인일,최초승인연도,공장연령,공장동,자치구_밀집도점수,제조시설면적_log,노후공장여부
0,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,2005-02-02,{'2005-02-02': ['']},NaN,{},NaN,NaN,창성동,0.037022,4.062510,False
1,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,2005-02-02,{'2005-02-02': ['']},NaN,{},NaN,NaN,창성동,0.037022,4.062510,False
2,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,2005-02-02,{'2005-02-02': ['']},NaN,{},NaN,NaN,창성동,0.037022,4.062510,False
3,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']},1998.0,27.0,통의동,0.037022,4.383401,True
4,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']},1998.0,27.0,통의동,0.037022,4.383401,True



## 6. ML Dataset ① 공장 리스크 예측용

**파일명:** `ml_factory_risk_seoul_2025_v1.csv`  
**단위:** 공장 1개 = 1행  
**목적:** 공장 단위 고위험/저위험 분류 (Binary Classification)

- Label: `label_high_risk`  
  - 리스크 점수 상위 25% 공장 = 1, 나머지 = 0
- 주요 Feature:
  - `feature_base_age` : 공장연령
  - `feature_base_area` / `feature_base_area_log`
  - `feature_base_gu_density` : 자치구 밀집도
  - `feature_agg_dong_factory_count` : 해당 동의 공장 수


In [7]:
from sklearn.preprocessing import MinMaxScaler

# 6-1. 동 단위 공장 수
dong_counts = (
    df_seoul
    .groupby([COL_SIGUNGU, "공장동"])
    .size()
    .to_frame("동_공장수")
    .reset_index()
)
df_seoul = df_seoul.merge(
    dong_counts,
    on=[COL_SIGUNGU, "공장동"],
    how="left"
)

# 6-2. 리스크 점수 계산을 위한 Feature 선택
risk_features = []
if "공장연령" in df_seoul.columns:
    risk_features.append("공장연령")
if "자치구_밀집도점수" in df_seoul.columns:
    risk_features.append("자치구_밀집도점수")
if COL_MANUF_AREA + "_log" in df_seoul.columns:
    risk_features.append(COL_MANUF_AREA + "_log")

if len(risk_features) == 0:
    raise ValueError("리스크 점수 계산에 사용할 Feature가 없습니다. 전처리를 확인해주세요.")

risk_df = df_seoul.dropna(subset=risk_features).copy()

scaler = MinMaxScaler()
scaled_cols = [c + "_scaled" for c in risk_features]
risk_df[scaled_cols] = scaler.fit_transform(risk_df[risk_features])

# 가중치 설정 (예시)
weights = {
    "공장연령": 0.4,
    "자치구_밀집도점수": 0.3,
    COL_MANUF_AREA + "_log": 0.3
}
w_list = [weights.get(f, 1.0 / len(risk_features)) for f in risk_features]

risk_df["risk_score"] = 0
for f, s, w in zip(risk_features, scaled_cols, w_list):
    risk_df["risk_score"] += w * risk_df[s]

# High risk 라벨 (상위 25%)
threshold_risk = risk_df["risk_score"].quantile(0.75)
risk_df["label_high_risk"] = (risk_df["risk_score"] >= threshold_risk).astype(int)
print("리스크 점수 상위 25% 기준값:", threshold_risk)

# 최종 ML Dataset 구성
ml1 = pd.DataFrame({
    "factory_id": risk_df[COL_FACTORY_ID] if COL_FACTORY_ID in risk_df.columns else risk_df.index,
    "company_name": risk_df[COL_COMPANY] if COL_COMPANY in risk_df.columns else None,
    "region_gu": risk_df[COL_SIGUNGU],
    "region_dong": risk_df["공장동"],
    "feature_base_age": risk_df["공장연령"] if "공장연령" in risk_df.columns else np.nan,
    "feature_base_area": risk_df[COL_MANUF_AREA] if COL_MANUF_AREA in risk_df.columns else np.nan,
    "feature_base_area_log": risk_df[COL_MANUF_AREA + "_log"] if COL_MANUF_AREA + "_log" in risk_df.columns else np.nan,
    "feature_base_gu_density": risk_df["자치구_밀집도점수"],
    "feature_agg_dong_factory_count": risk_df["동_공장수"],
    "risk_score": risk_df["risk_score"],
    "label_high_risk": risk_df["label_high_risk"],
})

ml1.to_csv(f"{PROCESSED_DATA_PATH}/ml_factory_risk_seoul_2025_v1.csv", index=False)
print("✅ ml_factory_risk_seoul_2025_v1.csv 저장 완료")
ml1.head()


리스크 점수 상위 25% 기준값: 0.4965806589263023
✅ ml_factory_risk_seoul_2025_v1.csv 저장 완료


,factory_id,company_name,region_gu,region_dong,feature_base_age,feature_base_area,feature_base_area_log,feature_base_gu_density,feature_agg_dong_factory_count,risk_score,label_high_risk
3,111100000000003,(주)에취.알.디,종로구,통의동,27.0,79.11,4.383401,0.037022,4,0.281215,0
4,111100000000003,(주)에취.알.디,종로구,통의동,27.0,79.11,4.383401,0.037022,4,0.281215,0
5,111100000000002,(주)코리아쉬핑가제트,종로구,통의동,27.0,92.40,4.536891,0.037022,4,0.286694,0
6,111100000000002,(주)코리아쉬핑가제트,종로구,통의동,27.0,92.40,4.536891,0.037022,4,0.286694,0
10,111100000000005,중앙인쇄사,종로구,적선동,27.0,55.00,4.025352,0.037022,5,0.268434,0



## 7. ML Dataset ② 자치구별 노후공장 비중

**파일명:** `ml_region_old_factory_share_seoul_2025_v1.csv`  
**단위:** 자치구 1개 = 1행  
**목적:** 자치구 단위로 **노후공장 비중이 높은 지역 vs 낮은 지역** 분류/분석

- Label: `label_high_old_share`  
  - 노후공장비율 상위 30% 자치구 = 1
- Feature 예시:
  - `feature_factory_count` : 공장 수
  - `feature_old_factory_count` : 노후공장 수
  - `feature_old_share` : 노후공장 비율
  - `feature_avg_age` : 평균 공장연령
  - `feature_avg_area_log` : 평균 면적 로그
  - `feature_dong_count` : 공장이 존재하는 동 개수


In [8]:

# 자치구별 동 개수
dong_per_gu = (
    df_seoul
    .groupby(COL_SIGUNGU)["공장동"]
    .nunique()
    .to_frame("feature_dong_count")
)

# 자치구 단위 집계
agg_dict = {
    "공장연령": "mean",
    "노후공장여부": "sum",
}

# 공장 수를 세기 위한 기준 컬럼 (식별자)
id_for_count = COL_FACTORY_ID if COL_FACTORY_ID in df_seoul.columns else df_seoul.columns[0]
agg_dict[id_for_count] = "count"

# 면적 로그 평균 추가
area_log_col = None
if COL_MANUF_AREA + "_log" in df_seoul.columns:
    agg_dict[COL_MANUF_AREA + "_log"] = "mean"
    area_log_col = COL_MANUF_AREA + "_log"

gu_agg = df_seoul.groupby(COL_SIGUNGU).agg(agg_dict)

gu_agg = gu_agg.rename(columns={
    id_for_count: "feature_factory_count",
    "노후공장여부": "feature_old_factory_count",
    "공장연령": "feature_avg_age"
})

if area_log_col:
    gu_agg = gu_agg.rename(columns={area_log_col: "feature_avg_area_log"})

gu_agg["feature_old_share"] = gu_agg["feature_old_factory_count"] / gu_agg["feature_factory_count"]

# 동 개수 merge
gu_agg = gu_agg.merge(dong_per_gu, left_index=True, right_index=True, how="left")

# Label: 노후공장비율 상위 30%
threshold_share = gu_agg["feature_old_share"].quantile(0.7)
gu_agg["label_high_old_share"] = (gu_agg["feature_old_share"] >= threshold_share).astype(int)
print("노후공장비율 상위 30% 기준값:", threshold_share)

ml2 = gu_agg.reset_index().rename(columns={COL_SIGUNGU: "region_gu"})

ml2.to_csv(f"{PROCESSED_DATA_PATH}/ml_region_old_factory_share_seoul_2025_v1.csv", index=False)
print("✅ ml_region_old_factory_share_seoul_2025_v1.csv 저장 완료")
ml2.head()


노후공장비율 상위 30% 기준값: 0.16328483098948676
✅ ml_region_old_factory_share_seoul_2025_v1.csv 저장 완료


,region_gu,feature_avg_age,feature_old_factory_count,feature_factory_count,feature_avg_area_log,feature_old_share,feature_dong_count,label_high_old_share
0,강남구,19.615385,14,561,4.310796,0.024955,14,0
1,강동구,27.000000,41,204,4.706014,0.200980,8,1
2,강북구,23.714286,23,240,4.603265,0.095833,5,0
3,강서구,22.194004,444,2031,4.655124,0.218612,25,1
4,관악구,21.285714,4,152,4.453256,0.026316,6,0



## 8. ML Dataset ③ 동 단위 입지 Hotspot / Location Score

**파일명:** `ml_location_score_candidate_sites_v1.csv`  
**단위:** `자치구 + 동` 조합 1개 = 1행  
**목적:** 공장이 많이 모여 있는 **제조업 Hotspot 동**을 찾고, 입지 스코어링에 활용

- Label: `label_hotspot`  
  - 공장 수 상위 25% 동 = 1
- Feature 예시:
  - `feature_factory_count` : 해당 동 공장 수
  - `feature_avg_age` : 평균 공장연령
  - `feature_old_share` : 노후공장 비율
  - `feature_avg_area_log` : 평균 면적 로그
  - `feature_gu_density` : 소속 자치구 밀집도 점수


In [9]:

# 동 단위 집계
agg_cols = {
    "공장연령": "mean",
    "노후공장여부": "mean",
}

id_for_count = COL_FACTORY_ID if COL_FACTORY_ID in df_seoul.columns else df_seoul.columns[0]
agg_cols[id_for_count] = "count"

if COL_MANUF_AREA + "_log" in df_seoul.columns:
    agg_cols[COL_MANUF_AREA + "_log"] = "mean"

dong_agg = (
    df_seoul
    .groupby([COL_SIGUNGU, "공장동"])
    .agg(agg_cols)
    .reset_index()
)

dong_agg = dong_agg.rename(columns={
    id_for_count: "feature_factory_count",
    "공장연령": "feature_avg_age",
    "노후공장여부": "feature_old_share"
})

if COL_MANUF_AREA + "_log" in df_seoul.columns:
    dong_agg = dong_agg.rename(columns={
        COL_MANUF_AREA + "_log": "feature_avg_area_log"
    })

# 자치구 밀집도 점수 merge
dong_agg = dong_agg.merge(
    df_seoul[[COL_SIGUNGU, "자치구_밀집도점수"]].drop_duplicates(),
    on=COL_SIGUNGU,
    how="left"
)
dong_agg = dong_agg.rename(columns={"자치구_밀집도점수": "feature_gu_density"})

# Hotspot 라벨: 공장 수 상위 25%
threshold_factory = dong_agg["feature_factory_count"].quantile(0.75)
dong_agg["label_hotspot"] = (dong_agg["feature_factory_count"] >= threshold_factory).astype(int)
print("동 단위 공장 수 상위 25% 기준값:", threshold_factory)

ml3 = dong_agg.rename(columns={
    COL_SIGUNGU: "region_gu",
    "공장동": "region_dong"
})

ml3.to_csv(f"{PROCESSED_DATA_PATH}/ml_location_score_candidate_sites_v1.csv", index=False)
print("✅ ml_location_score_candidate_sites_v1.csv 저장 완료")
ml3.head()


동 단위 공장 수 상위 25% 기준값: 38.75
✅ ml_location_score_candidate_sites_v1.csv 저장 완료


,region_gu,region_dong,feature_avg_age,feature_old_share,feature_factory_count,feature_avg_area_log,feature_gu_density,label_hotspot
0,강남구,개포동,NaN,0.0,46,4.365898,0.040508,1
1,강남구,논현동,15.0,0.0,95,4.291453,0.040508,1
2,강남구,대치동,12.0,0.0,64,4.334614,0.040508,1
3,강남구,도곡동,NaN,0.0,38,4.412931,0.040508,0
4,강남구,도곡제2동,NaN,0.0,3,3.895284,0.040508,0



## 9. 생성된 ML Dataset 요약 및 AIRD 관점 설명

이 노트북은 다음 3개의 ML Dataset을 생성했습니다.

1. **공장 리스크 예측용** – `ml_factory_risk_seoul_2025_v1.csv`  
   - 단위: 공장  
   - Label: `label_high_risk`  
   - 용도: 고위험 공장 탐지, 보험/안전 컨설팅 타깃 선정

2. **자치구 노후공장 비중 분석용** – `ml_region_old_factory_share_seoul_2025_v1.csv`  
   - 단위: 자치구  
   - Label: `label_high_old_share`  
   - 용도: 노후공장 집중 자치구 식별, 안전/환경 정책 우선순위 설정

3. **동 단위 입지 Hotspot 분석용** – `ml_location_score_candidate_sites_v1.csv`  
   - 단위: 자치구+동  
   - Label: `label_hotspot`  
   - 용도: 제조업 Hotspot 동 탐색, 산업입지/인프라/ESG 전략 수립

---

### 🔎 AIRD Pack 관점에서 이 노트북이 의미하는 것

- **원본 행정데이터 → 품질·정제 → ML Task별 Dataset 자동 생성**  
- 각 Dataset은
  - **관측단위 (factory / region / region_dong)**,
  - **Label 정의**,
  - **Feature 그룹(기본/집계/노후/밀집도)**  
  가 명확히 드러나도록 설계됨

이를 확장하면,

- 타 지역(수도권/전국)으로 확장
- 타 부처 데이터(안전, 환경, 에너지, 고용 등)와 결합
- Task를 세분화(사고예측, 민원예측, 투자성과예측 등)

하여, **범정부 AIRD ML Pack 표준**의 프로토타입으로 제시할 수 있습니다.
